# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,11.11,64,100,11.89,SH,1698604959
1,1,port mathurin,-19.6833,63.4167,24.57,79,12,6.72,MU,1698604959
2,2,olonkinbyen,70.9221,-8.7187,0.22,72,100,1.10,SJ,1698604959
3,3,port alfred,-33.5906,26.8910,14.54,75,99,1.34,ZA,1698604960
4,4,vadso,70.0744,29.7487,-3.91,80,75,4.12,NO,1698604960


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
coordinates = {
    "latitude": city_data_df["Lat"],
    "longitude": city_data_df["Lng"],
    "city": city_data_df["City"],
    "humidity": city_data_df["Humidity"]
}

coordinates_df = pd.DataFrame(coordinates)


# Display the map
map_plot_1 = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "humidity",
    scale = 0.6,
    color = "city")

map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (city,humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[city_data_df["Max Temp"] > 21, :]
ideal_city_df = ideal_city_df.loc[ideal_city_df["Max Temp"] < 28,:]
#ideal_city_df = ideal_city_df.loc[ideal_city_df["Cloudiness"] < 50,:]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna(how = 'any')

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,port mathurin,-19.6833,63.4167,24.57,79,12,6.72,MU,1698604959
5,5,uturoa,-16.7333,-151.4333,26.02,80,16,7.03,PF,1698604960
13,13,belmonte,-15.8631,-38.8828,26.61,64,40,5.09,BR,1698604962
15,15,coacotla,17.9333,-94.6750,26.86,64,15,2.55,MX,1698604963
23,23,lihue,21.9789,-159.3672,25.58,83,0,3.09,US,1698604675
...,...,...,...,...,...,...,...,...,...,...
557,557,boundiali,9.5217,-6.4869,26.44,49,0,1.23,CI,1698605140
558,558,bonthe,7.5264,-12.5050,26.44,84,28,3.67,SL,1698605140
561,561,pacasmayo,-7.4006,-79.5714,22.20,72,85,7.05,PE,1698605141
569,569,at taj,24.2049,23.2857,23.46,25,0,4.27,LY,1698605144


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel = {
    "city": ideal_city_df["City"],
    "country": ideal_city_df["Country"],
    "lat": ideal_city_df["Lat"],
    "lng": ideal_city_df["Lng"],
    "humidity": ideal_city_df["Humidity"]
}

hotel_df = pd.DataFrame(hotel)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["hotel name"] = ""

# Display sample data
hotel_df

,city,country,lat,lng,humidity,hotel name
1,port mathurin,MU,-19.6833,63.4167,79,
5,uturoa,PF,-16.7333,-151.4333,80,
13,belmonte,BR,-15.8631,-38.8828,64,
15,coacotla,MX,17.9333,-94.6750,64,
23,lihue,US,21.9789,-159.3672,83,
...,...,...,...,...,...,...
557,boundiali,CI,9.5217,-6.4869,49,
558,bonthe,SL,7.5264,-12.5050,84,
561,pacasmayo,PE,-7.4006,-79.5714,72,
569,at taj,LY,24.2049,23.2857,25,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index,"lat"]
    lng = hotel_df.loc[index,"lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    categories = "accommodation.hotel"
    radius = 10000
    filters = f"circle:{lng},{lat},{radius}"
    bias = f"proximity:{lng},{lat}"
    limit = 1

    params = {
        "categories":categories,
        "limit":limit,
        "filter":filters,
        "bias": bias,
        "apiKey":geoapify_key
    }
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    response = requests.get(base_url, params = params).json()
    
    # Convert the API response to JSON format
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "hotel name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "hotel name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'city']} - nearest hotel: {hotel_df.loc[index, 'hotel name']}")

# Display sample data
hotel_df

Starting hotel search
port mathurin - nearest hotel: Escale Vacances
uturoa - nearest hotel: Hawaiki Nui hotel
belmonte - nearest hotel: No hotel found
coacotla - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
adamstown - nearest hotel: No hotel found
sinabang - nearest hotel: No hotel found
phanom sarakham - nearest hotel: No hotel found
las palmas - nearest hotel: Era de La Corte
kununurra - nearest hotel: Kununurra Country Club Resort
sinamar - nearest hotel: No hotel found
hithadhoo - nearest hotel: Pebbles Inn
antsiranana - nearest hotel: Central
georgetown - nearest hotel: Page 63 hostel
wanning - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
takoradi - nearest hotel: Hillcrest
salinas - nearest hotel: The Traveler's Hotel
impfondo - nearest hotel: Tropicana
flying fish cove - nearest hotel: Christmas Island Lodge
prado - nearest hotel: Pousada Recanto do Prado
sijunjung - nearest hotel: Hotel Bukik Gadang
balikpapan - nearest

,city,country,lat,lng,humidity,hotel name
1,port mathurin,MU,-19.6833,63.4167,79,Escale Vacances
5,uturoa,PF,-16.7333,-151.4333,80,Hawaiki Nui hotel
13,belmonte,BR,-15.8631,-38.8828,64,No hotel found
15,coacotla,MX,17.9333,-94.6750,64,No hotel found
23,lihue,US,21.9789,-159.3672,83,Kauai Palms
...,...,...,...,...,...,...
557,boundiali,CI,9.5217,-6.4869,49,No hotel found
558,bonthe,SL,7.5264,-12.5050,84,No hotel found
561,pacasmayo,PE,-7.4006,-79.5714,72,Playa Linda
569,at taj,LY,24.2049,23.2857,25,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
coordinates = {
    "latitude": hotel_df["lat"],
    "longitude": hotel_df["lng"],
    "city": hotel_df["city"],
    "humidity": hotel_df["humidity"],
    "hotel name": hotel_df["hotel name"],
    "country": hotel_df["country"]
}

coordinates_df = pd.DataFrame(coordinates)

map_plot_2 = coordinates_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "humidity",
    scale = 0.6,
    color = "city",
    hover_cols = ["hotel name", "country"])

# Display the map
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [longitude,latitude]   (city,humidity,hotel name,country)